<a href="https://colab.research.google.com/github/RaymondGarcia1107/HighlyCaffeinated/blob/main/Sales_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Sales Predictions - Part 1
Raymond Garcia

## 1- Importing Libraries

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2- Loading Data

In [2]:
sales = pd.read_csv('/content/drive/MyDrive/Datafiles/sales_predictions.csv')
salescopy = sales.copy()
sales.info()
sales.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## 3- Data Cleaning
*  The data has 8532 rows and 12 columns. Outlet_Establishment_Year seems to be an integer, but its a date-part. I considered changing it to a category, but this will complicate filtering, so I left it as an integer. The rest of the data types seem fine for the first few records. 

### 3.1- Duplicates
First I will check for duplicates and work through those

In [3]:
sales.duplicated().sum()

0

*  There are no duplicates in the data.



### 3.2- Missing Values
Next I will look for missing values and handle those.

In [4]:
sales.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

*  We have 1463 empty values for Item_Weight and 2410 for Outlet_Size. Both of these are over 17% of the total dataset, so we will not drop the rows. Instead I will look to impute the missing values for those rows.

#### 3.2.1- Item_Weight
I'll start with Item_Weight

In [5]:
items_no_weight = sales.loc[sales['Item_Weight'].isnull(),'Item_Identifier']
items_no_weight = items_no_weight.drop_duplicates()
items_no_weight

7       FDP10
18      DRI11
21      FDW12
23      FDC37
29      FDC14
        ...  
8463    FDG59
8472    NCS41
8480    FDQ58
8487    DRG13
8504    NCN18
Name: Item_Identifier, Length: 1142, dtype: object

*  This is a series of all the Item_Identifiers that do not have a weight. I will use this to check if the item has a weight on another row and change the values to that value. Because the Item_Identifier is a unique ID, I can assume that the weights for each ID should be the same, but I will verify that below.

In [6]:
no_weight = sales.merge(items_no_weight, on = 'Item_Identifier', how = 'inner')
no_weight = no_weight.dropna(subset = ['Item_Weight']).groupby('Item_Identifier')['Item_Weight'].agg(['min','max'])
no_weight['diff'] = no_weight['min'] - no_weight['max']
no_weight.loc[no_weight['diff'] != 0,:].shape

(0, 3)

*  Because there was no difference between the min and max, we can replace all the null rows for Item_weight with the min value in the dataframe.

In [7]:
no_weight = no_weight.loc[:,'min']
sales = sales.merge(no_weight, on = 'Item_Identifier', how = 'left')
sales['Item_Weight'].fillna(sales['min'], inplace = True)
sales.drop(columns = ['min'], inplace= True)
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.6+ KB


*  We have successfully removed the nulls from Item_Weight, leaving 4 rows behind. Since this is a small subset of the data, we can drop these rows. 

In [8]:
sales.dropna(subset = ['Item_Weight'],inplace = True)
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8519 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8519 non-null   object 
 3   Item_Visibility            8519 non-null   float64
 4   Item_Type                  8519 non-null   object 
 5   Item_MRP                   8519 non-null   float64
 6   Outlet_Identifier          8519 non-null   object 
 7   Outlet_Establishment_Year  8519 non-null   int64  
 8   Outlet_Size                6109 non-null   object 
 9   Outlet_Location_Type       8519 non-null   object 
 10  Outlet_Type                8519 non-null   object 
 11  Item_Outlet_Sales          8519 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.2+ KB


#### 3.2.2- Outlet_Size
I'll now tackle the Outlet_Size column

In [9]:
items_no_outlet = sales.loc[sales['Outlet_Size'].isnull(),'Outlet_Identifier']
print(f'Items without a size make up {items_no_outlet.shape[0]} rows')
items_no_outlet = items_no_outlet.drop_duplicates()
print(f"There are a total of {items_no_outlet.shape[0]} Unique Outlet ID's")

Items without a size make up 2410 rows
There are a total of 3 Unique Outlet ID's


In [10]:
no_size = sales.merge(items_no_outlet, on = 'Outlet_Identifier', how = 'inner')
no_size.dropna()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


*  When going through the same exercise for Outlet_Size, we found that the 3 outlets, which make up 2410 rows, do not have an entry for Outlet_Size. Because its Categorical, we can set the value of the null rows to "Unknown" for now.

In [11]:
sales['Outlet_Size'].fillna('Unknown', inplace = True)
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8519 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8519 non-null   object 
 3   Item_Visibility            8519 non-null   float64
 4   Item_Type                  8519 non-null   object 
 5   Item_MRP                   8519 non-null   float64
 6   Outlet_Identifier          8519 non-null   object 
 7   Outlet_Establishment_Year  8519 non-null   int64  
 8   Outlet_Size                8519 non-null   object 
 9   Outlet_Location_Type       8519 non-null   object 
 10  Outlet_Type                8519 non-null   object 
 11  Item_Outlet_Sales          8519 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.2+ KB


### 3.3- Data Normalization
*  Now I want to take a look at the objects and normalize them. I want to look at:
  *  Item_Fat_Content
  *  Item_Type
  *  Outlet_Location_Type
  *  Outlet_Type

In [12]:
sales.value_counts(subset = ['Item_Fat_Content'])

Item_Fat_Content
Low Fat             5088
Regular             2886
LF                   316
reg                  117
low fat              112
dtype: int64

In [13]:
sales['Item_Fat_Content'].replace(['LF','reg','low fat'],['Low Fat','Regular','Low Fat'],inplace = True)
sales.value_counts(subset = 'Item_Fat_Content')

Item_Fat_Content
Low Fat    5516
Regular    3003
dtype: int64

In [14]:
sales.value_counts(subset = ['Item_Type'])

Item_Type            
Fruits and Vegetables    1232
Snack Foods              1199
Household                 910
Frozen Foods              855
Dairy                     681
Canned                    649
Baking Goods              647
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
dtype: int64

In [15]:
sales.value_counts(subset = ['Outlet_Location_Type'])

Outlet_Location_Type
Tier 3                  3347
Tier 2                  2785
Tier 1                  2387
dtype: int64

In [16]:
sales.value_counts(subset=['Outlet_Type'])

Outlet_Type      
Supermarket Type1    5577
Grocery Store        1082
Supermarket Type3     932
Supermarket Type2     928
dtype: int64

*  These datatypes look good!

### 3.4- Quick Descriptive Statistics
*  Finally we will look at some descriptive statistics for each of the numeric columns

In [17]:
sales.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8519.000000,8519.000000,8519.000000,8519.000000,8519.000000
mean,12.875420,0.066112,141.010019,1997.837892,2181.188779
std,4.646098,0.051586,62.283594,8.369105,1706.511093
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026983,93.844900,1987.000000,834.247400
50%,12.650000,0.053925,143.047000,1999.000000,1794.331000
75%,16.850000,0.094558,185.676600,2004.000000,3100.630600
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


## Exploratory Visuals

## Explanatory Visuals